In [2]:
from core.utils import Tibanna
from dcicutils import ff_utils
from core.utils import run_workflow
from datetime import datetime
import time

env = 'data'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key={"default" : tibanna.ff_keys})

intr = "intermediate file"

Converter = {
    "bwa-mem 0.2.5":{"bam":intr},
    "hi-c-processing-bam 0.2.5":{"bam": "alignment",
                                 "pairs": 'contact list-replicate'},
    "hi-c-processing-pairs 0.2.5":{"normvector_juicerformat": "juicebox norm vector",
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"},
    "hi-c-processing-pairs-nore 0.2.5":{"normvector_juicerformat": "juicebox norm vector",
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"},
    "hi-c-processing-pairs-nonorm 0.2.5":{
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"},
    "hi-c-processing-pairs-nore-nonorm 0.2.5":{
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"}
    
}



my_url = '/search/?sort=-date_created&type=WorkflowRunAwsem&limit=all'
all_aws = ff_utils.search_metadata(my_url, ff_env=env)
print len(all_aws)

6509


In [4]:
my_wfrs = Converter.keys()
print my_wfrs

out_n = "This is an output file of the Hi-C processing pipeline"
int_n = "This is an intermediate file in the HiC processing pipeline"

success = 0
patch_problem = 0
deleted = 0
for wfr_resp in all_aws:
    if wfr_resp['status'] == 'deleted':
        deleted +=1
        continue
        
    wfr_name = wfr_resp['display_title'].split(' run')[0]
    if wfr_name in my_wfrs:
        outputs = wfr_resp.get('output_files')
        for output in outputs:
            print output
            break
            form = output['format']
            file_id = output['value']
            f_type =  Converter[wfr_name][form]
            
            print form
            print file_id
            print f_type
            
            desc = ""
            if f_type == "intermediate file":
                desc = int_n
            else:
                desc = out_n
                
            patch_data = {"description": desc, 'file_type': f_type}
            try:
                #print file_id
                #print(ff_utils.get_metadata(file_id, connection=ff)['uuid'])
                #ff_utils.patch_metadata(patch_data, obj_id=file_id ,connection=ff)
                success += 1
                print success
            except Exception as e:
                print e
                print
                patch_problem += 1
    break
            
print success
print patch_problem
print deleted


['hi-c-processing-pairs-nonorm 0.2.5', 'hi-c-processing-pairs 0.2.5', 'hi-c-processing-pairs-nore 0.2.5', 'hi-c-processing-bam 0.2.5', 'bwa-mem 0.2.5', 'hi-c-processing-pairs-nore-nonorm 0.2.5']
{u'secondary_file_formats': [u'pairs_px2'], u'extension': u'.pairs.gz', u'format': u'pairs', u'workflow_argument_name': u'merged_pairs', u'value': {u'uuid': u'd3b89043-e103-4284-929f-761929085f9d', u'file_format': u'pairs', u'display_title': u'4DNFIIUM9A7W.pairs.gz', u'accession': u'4DNFIIUM9A7W', u'link_id': u'~files-processed~4DNFIIUM9A7W~', u'principals_allowed': {u'audit': [u'system.Everyone'], u'edit': [u'group.admin', u'submits_for.828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'], u'view': [u'award.b0b9c607-f8b4-4f02-93f4-9895b461334b', u'group.admin', u'group.read-only-admin', u'lab.828cd4fe-ebb0-4b36-a94a-d2e3a36cc989', u'remoteuser.EMBED', u'remoteuser.INDEXER']}, u'file_size': 2933761234, u'@id': u'/files-processed/4DNFIIUM9A7W/', u'@type': [u'FileProcessed', u'File', u'Item']}, u'upload_key': 